### NPSC psudeo-extension stands for NuPlan Scenario Collection. 
#### It is a pkl file with the naming convention `filename.npsc.pkl` that contains:
* The data structure of a dictionary of scenario modifications.
* The `DATASET_PARAMS` list of filters in order to extract the correct scenarios to modify.

In [1]:
import logging
import random
import nest_asyncio
import pandas as pd

nest_asyncio.apply()

import torch

torch.cuda.is_available()

import os
from pathlib import Path
import tempfile

from shutil import rmtree
from typing import List, Optional, Union

from nuplan.planning.script.run_simulation import main as main_simulation

In [2]:
%env NUPLAN_DATA_ROOT=../../../datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=../../data/nuplan/maps
%env NUPLAN_EXP_ROOT=../../data/nuplan/exp
%env NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/
%pwd

env: NUPLAN_DATA_ROOT=../../../datasets/nuplan/dataset
env: NUPLAN_MAPS_ROOT=../../data/nuplan/maps
env: NUPLAN_EXP_ROOT=../../data/nuplan/exp
env: NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/


'/home/ehdykhne/nuplan-devkit/experiments'

In [3]:
import hydra
import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf

from nuplan.common.utils.s3_utils import is_s3_path
from nuplan.planning.script.builders.simulation_builder import build_simulations
from nuplan.planning.script.builders.simulation_callback_builder import (
    build_callbacks_worker,
    build_simulation_callbacks,
)
from nuplan.planning.script.utils import (
    run_runners,
    set_default_path,
    set_up_common_builder,
)
from nuplan.planning.simulation.planner.abstract_planner import AbstractPlanner

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# If set, use the env. variable to overwrite the default dataset and experiment paths
set_default_path()

# If set, use the env. variable to overwrite the Hydra config
CONFIG_PATH = os.getenv("NUPLAN_HYDRA_CONFIG_PATH", "config/simulation")

if os.environ.get("NUPLAN_HYDRA_CONFIG_PATH") is not None:
    CONFIG_PATH = os.path.join("../../../../", CONFIG_PATH)

if os.path.basename(CONFIG_PATH) != "simulation":
    CONFIG_PATH = os.path.join(CONFIG_PATH, "simulation")
CONFIG_NAME = "default_simulation"

In [4]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(
    os.getenv("NUPLAN_TUTORIAL_PATH", ""), "../nuplan/planning/script"
)
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

In [8]:
import pickle

experiment_name = "base_bench"
modification_file_path = "npsc_files/" + experiment_name + ".npsc.pkl"
with open(modification_file_path, "rb") as output_file:
    mod_list_dict = pickle.load(output_file)
    print(list(mod_list_dict.keys()))
    DATASET_PARAMS = pickle.load(output_file)

print(DATASET_PARAMS)

['f3610d9738ef589b', '8bcd750101ab51fc']
['scenario_builder=val', 'scenario_filter=all_scenarios', "scenario_filter.scenario_types=['starting_left_turn', 'starting_right_turn', 'starting_straight_stop_sign_intersection_traversal', 'starting_straight_traffic_light_intersection_traversal', 'starting_unprotected_cross_turn', 'starting_unprotected_noncross_turn', 'traversing_intersection', 'traversing_traffic_light_intersection']", 'scenario_filter.ego_displacement_minimum_m=10', 'scenario_filter.map_names=[sg-one-north, us-ma-boston, us-pa-pittsburgh-hazelwood, us-nv-las-vegas-strip]', 'scenario_filter.num_scenarios_per_type=400']


In [11]:
urban_ckpt = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/urbandriver_checkpoint.ckpt"
gc_pgp_ckpt = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/gc_pgp_checkpoint.ckpt"
hybrid_ckpt = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/pdm_offset_checkpoint.ckpt"
#'/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/urbandriver_checkpoint.ckpt'
#'/home/ehdykhne/Repos/nuplan-devkit/experiments/pretrained_checkpoints/urbandriver_checkpoint.ckpt'
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
print(simulation_hydra_paths.config_name)
cfg_occ = hydra.compose(
    config_name=simulation_hydra_paths.config_name,
    overrides=[
        # planner
        ################################################
        "planner=pdm_closed_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
        # planner occlusion
        ################################################
        "+occlusion_cfg.occlusion=true",
        "+occlusion_cfg.manager_type=wedge",  # options: [range, shadow, wedge]
        "+occlusion_cfg.uncloak_reaction_time=1.5",  # 0.1
        "+occlusion_cfg.notice_threshold=1.0",  # 0.1
        ################################################
        # obs
        ################################################
        f"observation.planner_type=pdm_closed",
        ################################################
        # obs occlusion
        ################################################
        f"observation.occlusion_cfg.occlusion=True",
        f"observation.occlusion_cfg.manager_type=wedge",
        f"+observation.occlusion_cfg.uncloak_reaction_time=1.5",
        f"+observation.occlusion_cfg.notice_threshold=1.0",
        ################################################
        # scenario modification
        "+modify_scenario_simulations=true",
        "+modifier_types=[diverge-conflict-with-occlusion-injection]",  # can select multiple: [occlusion-injection, left-and-right,
        # oncoming-left-turn-and-occlusion-injection, conflict-and-occlusion-injection,
        # sequential-conflict-with-occlusion-injection, cross-conflict-with-occlusion-injection
        # merge-conflict-with-occlusion-injection, diverge-conflict-with-occlusion-injection]
        "+second_testing_round=true",
        f"+modification_file_path={modification_file_path}",
        "+remove_other_agents=true",
        ################################################
        # misc
        ################################################
        "+simulation=closed_loop_multiagent",  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents, closed_loop_multiagent]
        "worker=ray_distributed",  # [sequential, ray_distributed]
        "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
        *DATASET_PARAMS,
        ################################################
    ],
)

output_folder_occ = cfg_occ.output_dir
print('output_folder_occ = "' + output_folder_occ + '"')

default_simulation
output_folder_occ = "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.04.14.05.22.36"


### Now, we run the simulations

In [12]:
# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
main_simulation(cfg_occ)

Global seed set to 0
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback...
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback: 4...DONE!


2024-04-14 05:23:20,642 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2024-04-14 05:23:20,680 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_ray.py:78}  Starting ray local!


2024-04-14 05:23:22,410	INFO worker.py:1636 -- Started a local Ray instance.


2024-04-14 05:23:23,292 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: RayDistributed
2024-04-14 05:23:23,292 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 32
Number of GPUs per node: 4
Number of threads across all nodes: 32
2024-04-14 05:23:23,292 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2024-04-14 05:23:23,293 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:32}  Building experiment folders...
2024-04-14 05:23:23,293 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:35}  

	Folder where all results are stored: ../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.04.14.05.22.36

2024-04-14 05:23:23,295 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_build

Ray objects: 100%|██████████| 32/32 [00:09<00:00,  3.45it/s]
(raylet) [2024-04-14 05:23:42,449 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613494910976; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:23:52,459 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613443186688; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:24:02,470 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613438676992; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:24:12,475 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is ov

...

(raylet) [2024-04-14 05:24:32,487 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613420179456; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:24:42,493 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613421281280; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:24:52,499 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613414973440; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:25:02,508 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613413076992; capacity: 1612571

2024-04-14 05:25:06,966 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:151}  Modyfing Scenarios...
[
	'8bcd750101ab51fc',
	'f3610d9738ef589b'
]
2024-04-14 05:25:06,991 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:206}  Created 2 modified scenarios from 2689 scenarios, 2 of which were modifiable.
2024-04-14 05:25:06,993 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:207}  Building simulations...DONE!
2024-04-14 05:25:06,993 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/run_simulation.py:113}  Running simulation...
2024-04-14 05:25:06,993 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/utils.py:138}  Executing runners...
2024-04-14 05:25:06,994 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py:82}  Starting 2 simulations using RayDistributed!


Ray objects:   0%|          | 0/2 [00:00<?, ?it/s](raylet) [2024-04-14 05:25:12,516 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613408944128; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:25:22,522 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613357989888; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:25:32,528 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613302308864; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:25:42,535 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full

2024-04-14 05:25:52,936 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py:129}  Number of successful simulations: 2
2024-04-14 05:25:52,937 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py:130}  Number of failed simulations: 0
2024-04-14 05:25:52,937 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/utils.py:147}  Finished executing runners!
2024-04-14 05:25:52,955 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/utils.py:74}  Saved runner reports to ../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.04.14.05.22.36/runner_report.parquet
2024-04-14 05:25:52,955 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/time_callback.py:27}  Simulation duration: 00:02:32 [HH:MM:SS]
2024-04-14 05:25:53,037 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/metric_file_callback.py:79}  Metric files integration: 00:00:00 [HH:MM:SS]
2024-04-14 05:25:53,105 I

(raylet) [2024-04-14 05:26:02,548 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613262786560; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:26:12,561 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613258465280; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:26:22,572 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613254770688; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:26:32,576 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 613250539520; capacity: 1612571

In [15]:
import re

output_folder = [output_folder_occ]

print(output_folder_occ)

CONFIG_PATH = "../nuplan/planning/script/config/nuboard"
CONFIG_NAME = "default_nuboard"

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)
scenario_builder = None
for item in DATASET_PARAMS:
    parts = re.split("=", item, 1)
    if parts[0] == "scenario_builder":
        scenario_builder = parts[1]
if scenario_builder is None:
    raise ValueError("scenario_builder not specified in the dataset parameters")
print(scenario_builder)
# Compose the configuration
cfg = hydra.compose(
    config_name=CONFIG_NAME,
    overrides=[
        f"scenario_builder={scenario_builder}",  # set the database (same as simulation) used to fetch data for visualization
        f"simulation_path={output_folder}",  # [output_folder, output_folder_alt] nuboard file path(s), if left empty the user can open the file inside nuBoard
    ],
)

../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.04.14.05.22.36
val


### Launch NuBoard

In [16]:
from nuplan.planning.script.run_nuboard import main as main_nuboard

# Run nuBoard
main_nuboard(cfg)

2024-04-14 05:54:11,060 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:18}  Building AbstractScenarioBuilder...
2024-04-14 05:54:11,077 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/scenario_building_builder.py:21}  Building AbstractScenarioBuilder...DONE!
2024-04-14 05:54:11,078 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/nuboard.py:84}  Opening Bokeh application on http://localhost:5006/
2024-04-14 05:54:11,078 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/nuboard.py:85}  Async rendering is set to: True
2024-04-14 05:54:11,078 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/server.py:403}  Starting Bokeh server version 2.4.3 (running on Tornado 6.3.3)
2024-04-14 05:54:11,079 WARNING {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/util.py:145}  Host wildcard '*' will allow connections originating from multiple (or possib

INFO:tornado.access:200 GET / (127.0.0.1) 247.46ms
INFO:tornado.access:304 GET /resource/spectre.min.css (127.0.0.1) 2.59ms
INFO:tornado.access:304 GET /resource/spectre-exp.min.css (127.0.0.1) 0.33ms
INFO:tornado.access:304 GET /resource/spectre-icons.min.css (127.0.0.1) 0.26ms
INFO:tornado.access:304 GET /resource/style.css (127.0.0.1) 0.24ms
INFO:tornado.access:304 GET /resource/css/overview.css (127.0.0.1) 0.23ms
INFO:tornado.access:304 GET /resource/css/cloud.css (127.0.0.1) 0.25ms
INFO:tornado.access:304 GET /resource/css/scenario.css (127.0.0.1) 0.26ms
INFO:tornado.access:304 GET /resource/css/histogram.css (127.0.0.1) 0.67ms
INFO:tornado.access:304 GET /resource/scripts/utils.js (127.0.0.1) 0.24ms
INFO:tornado.access:304 GET /resource/motional_logo.png (127.0.0.1) 6.69ms


2024-04-14 05:54:12,081 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  304 GET /resource/css/histogram.css (127.0.0.1) 0.67ms
2024-04-14 05:54:12,088 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  304 GET /resource/scripts/utils.js (127.0.0.1) 0.24ms
2024-04-14 05:54:12,096 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  304 GET /resource/motional_logo.png (127.0.0.1) 6.69ms
2024-04-14 05:54:12,757 WARNING {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  404 GET /favicon.ico (127.0.0.1) 0.37ms
2024-04-14 05:54:12,798 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  101 GET /ws (127.0.0.1) 0.48ms
2024-04-14 05:54:12,798 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/bokeh/server/views/ws.py:132}  WebSocket connection opened
2024-04-14 05:54:

INFO:tornado.access:101 GET /ws (127.0.0.1) 0.48ms


2024-04-14 05:54:13,296 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/base/simulation_tile.py:172}  Minimum frame time=0.017 s
2024-04-14 05:54:13,302 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/tabs/scenario_tab.py:485}  Rending scenario plot takes 0.0007 seconds.
2024-04-14 05:54:13,401 INFO {/home/ehdykhne/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  200 GET / (127.0.0.1) 231.61ms


INFO:tornado.access:200 GET / (127.0.0.1) 231.61ms
(raylet) [2024-04-14 05:54:13,481 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 611385790464; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:54:23,487 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 611381190656; capacity: 16125711347712. Object creation will fail if spilling is required.
Rendering a scenario: 100%|██████████| 1/1 [00:00<00:00, 36.70it/s]


2024-04-14 05:54:26,371 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/nuboard/tabs/scenario_tab.py:485}  Rending scenario plot takes 3.1397 seconds.


(raylet) [2024-04-14 05:54:33,492 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 611373592576; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:54:43,497 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 611373805568; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:54:53,501 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 611369689088; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 05:55:03,506 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 611365462016; capacity: 1612571

KeyboardInterrupt: 

(raylet) [2024-04-14 06:06:03,859 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 610764320768; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 06:06:13,866 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 610760298496; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 06:06:23,869 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 610755166208; capacity: 16125711347712. Object creation will fail if spilling is required.
(raylet) [2024-04-14 06:06:33,873 E 555796 555808] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-04-14_05-23-20_714759_487069 is over 95% full, available space: 610703589376; capacity: 1612571